In [25]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
from tqdm import tqdm
import math
from scipy.spatial import distance

warnings.filterwarnings('ignore')


In [3]:
def bilinear(img, ax=1., ay=1.):
	if len(img.shape) == 2:
		H,W = img.shape
		dim = 2
	else:
		H, W, C = img.shape
		dim = 3

	aH = int(ay * H)
	aW = int(ax * W)

	# position of resized image
	y = np.arange(aH).repeat(aW).reshape(aH, -1)
	x = np.tile(np.arange(aW), (aH, 1))

	# position of original position
	y = (y / ay)
	x = (x / ax)

	ix = np.floor(x).astype(np.int)
	iy = np.floor(y).astype(np.int)

	ix = np.minimum(ix, W-2)
	iy = np.minimum(iy, H-2)

	# distance 
	dx = x - ix
	dy = y - iy

	dx = np.repeat(np.expand_dims(dx, axis=-1), 3, axis=-1)
	dy = np.repeat(np.expand_dims(dy, axis=-1), 3, axis=-1)
	
	def weight(x,y):
		return x*y

	# interpolate
	out = weight((1-dx),(1-dy)) * img[iy, ix] + weight(dx,(1 - dy)) * img[iy, ix+1] + weight((1 - dx),dy) * img[iy+1, ix] + weight(dx,dy) * img[iy+1, ix+1]
	out = np.clip(out, 0, 255)
	out = out.astype(np.uint8)

	return out



In [4]:
def bicubic_mod(img, ax=1., ay=1.):
	if len(img.shape) == 2:
		H,W = img.shape
		dim = 2
	else:
		H, W, C = img.shape
		dim = 3

	aH = int(ay * H)
	aW = int(ax * W)

	# position of resized image
	y = np.arange(aH).repeat(aW).reshape(aH, -1)
	x = np.tile(np.arange(aW), (aH, 1))

	# position of original position
	y = (y / ay)
	x = (x / ax)

	ix = np.floor(x).astype(np.int)
	iy = np.floor(y).astype(np.int)

	ix = np.minimum(ix, W-2)
	iy = np.minimum(iy, H-2)

	# distance 
	dx = x - ix
	dy = y - iy

	dx = np.repeat(np.expand_dims(dx, axis=-1), dim, axis=-1)
	dy = np.repeat(np.expand_dims(dy, axis=-1), dim, axis=-1)
	
	def weight(x,y):
		return x**2 * x**2 * (9 - 6*x - 6*y + 4*x*y)

	# interpolate
	out = weight((1-dx),(1-dy)) * img[iy, ix] + weight(dx,(1 - dy)) * img[iy, ix+1] + weight((1 - dx),dy) * img[iy+1, ix] + weight(dx,dy) * img[iy+1, ix+1]
	out = np.clip(out, 0, 255)
	out = out.astype(np.uint8)

	return out



In [5]:
def bicubic(img, ax=1., ay=1.):
	if len(img.shape) == 2:
		H,W = img.shape
		dim = 2
	else:
		H, W, C = img.shape
		dim = 3

	aH = int(ay * H)
	aW = int(ax * W)

	# positions of resized image
	y = np.arange(aH).repeat(aW).reshape(aH, -1)
	x = np.tile(np.arange(aW), (aH, 1))
	y = (y / ay)
	x = (x / ax)

	# positions of original image
	ix = np.floor(x).astype(np.int)
	iy = np.floor(y).astype(np.int)

	ix = np.minimum(ix, W-1)
	iy = np.minimum(iy, H-1)

	# distance of each position of original image
	dx2 = x - ix
	dy2 = y - iy
	dx1 = dx2 + 1
	dy1 = dy2 + 1
	dx3 = 1 - dx2
	dy3 = 1 - dy2
	dx4 = 1 + dx3
	dy4 = 1 + dy3

	dxs = [dx1, dx2, dx3, dx4]
	dys = [dy1, dy2, dy3, dy4]

	def weight(t):
		a = -1.
		at = np.abs(t)
		w = np.zeros_like(t)
		ind = np.where(at <= 1)
		w[ind] = ((a+2) * np.power(at, 3) - (a+3) * np.power(at, 2) + 1)[ind]
		ind = np.where((at > 1) & (at <= 2))
		w[ind] = (a*np.power(at, 3) - 5*a*np.power(at, 2) + 8*a*at - 4*a)[ind]
		return w

	w_sum = np.zeros((aH, aW, C), dtype=np.float32)
	out = np.zeros((aH, aW, C), dtype=np.float32)

	# interpolate
	for j in range(-1, 3):
		for i in range(-1, 3):
			ind_x = np.minimum(np.maximum(ix + i, 0), W-1)
			ind_y = np.minimum(np.maximum(iy + j, 0), H-1)

			wx = weight(dxs[i+1])
			wy = weight(dys[j+1])
			wx = np.repeat(np.expand_dims(wx, axis=-1), 3, axis=-1)
			wy = np.repeat(np.expand_dims(wy, axis=-1), 3, axis=-1)

			w_sum += wx * wy
			out += wx * wy * img[ind_y, ind_x]

	out /= w_sum
	out = np.clip(out, 0, 255)
	out = out.astype(np.uint8)

	return out

In [68]:
test_images = os.listdir("./test_images/")
ratio = 1.5
d1, d2, d3, d4, d5, d6, d7 = [], [], [], [], [], [], []

for __ in tqdm(test_images):
    raw = Image.open(f"./test_images/{__}")
    img = np.asarray(raw, dtype=np.float)
    if img.shape[0]%2 != 0 or img.shape[1]%2 != 0:
        continue
    img1 = bilinear(img, ax=ratio, ay=ratio)
    img2 = bicubic_mod(img, ax=ratio, ay=ratio)
    img3 = bicubic(img, ax=ratio, ay=ratio)
    img4 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.NEAREST)
    img5 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.BILINEAR)
    img6 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.BICUBIC)
    img7 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.LANCZOS)
    img1 = bilinear(img1, ax=1/ratio, ay=1/ratio)
    img2 = bicubic_mod(img2, ax=1/ratio, ay=1/ratio)
    img3 = bicubic(img3, ax=1/ratio, ay=1/ratio)
    img4 = raw.resize((img.shape[1], img.shape[0]), Image.NEAREST)
    img5 = raw.resize((img.shape[1], img.shape[0]), Image.BILINEAR)
    img6 = raw.resize((img.shape[1], img.shape[0]), Image.BICUBIC)
    img7 = raw.resize((img.shape[1], img.shape[0]), Image.LANCZOS)
    d1.append(distance.euclidean(img.flatten(), img1.flatten()))
    d2.append(distance.euclidean(img.flatten(), img2.flatten()))
    d3.append(distance.euclidean(img.flatten(), img3.flatten()))
    d4.append(distance.euclidean(img.flatten(), np.asarray(img4).flatten()))
    d5.append(distance.euclidean(img.flatten(), np.asarray(img5).flatten()))
    d6.append(distance.euclidean(img.flatten(), np.asarray(img6).flatten()))
    d7.append(distance.euclidean(img.flatten(), np.asarray(img7).flatten()))
    

100%|██████████| 50/50 [00:27<00:00,  1.79it/s]


In [69]:
print(sum(d1)/len(d1))
print(sum(d2)/len(d1))
print(sum(d3)/len(d1))
print(sum(d4)/len(d1))
print(sum(d5)/len(d1))
print(sum(d6)/len(d1))
print(sum(d7)/len(d1))

2920.759013747408
85006.91864039103
1128.200637546219
0.0
0.0
0.0
0.0


In [75]:
import time
raw =Image.open(f"./test_images/ILSVRC2012_test_00000193.JPEG")
img = np.asarray(raw, dtype=np.float)
print(img.shape)
start_time = time.time()
img1 = bilinear(img, ax=ratio, ay=ratio)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img2 = bicubic_mod(img, ax=ratio, ay=ratio)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img3 = bicubic(img, ax=ratio, ay=ratio)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img4 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.NEAREST)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img5 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.BILINEAR)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img6 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.BICUBIC)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img7 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.LANCZOS)
print("--- %s seconds ---" % (time.time() - start_time))

img1 = bilinear(img1, ax=1/ratio, ay=1/ratio)
img2 = bicubic_mod(img2, ax=1/ratio, ay=1/ratio)
img3 = bicubic(img3, ax=1/ratio, ay=1/ratio)
img4 = raw.resize((img.shape[1], img.shape[0]), Image.NEAREST)
img5 = raw.resize((img.shape[1], img.shape[0]), Image.BILINEAR)
img6 = raw.resize((img.shape[1], img.shape[0]), Image.BICUBIC)
img7 = raw.resize((img.shape[1], img.shape[0]), Image.LANCZOS)
print(distance.euclidean(img.flatten(), img1.flatten()))
print(distance.euclidean(img.flatten(), img2.flatten()))
print(distance.euclidean(img.flatten(), img3.flatten()))
print(distance.euclidean(img.flatten(), np.asarray(img4).flatten()))
print(distance.euclidean(img.flatten(), np.asarray(img5).flatten()))
print(distance.euclidean(img.flatten(), np.asarray(img6).flatten()))
print(distance.euclidean(img.flatten(), np.asarray(img7).flatten()))
Image.fromarray(img1).save("bilinear.png")
Image.fromarray(img2).save("bicubic_mod.png")
Image.fromarray(img3).save("bicubic.png")
img4.save("PIL-Nearest.png")
img5.save("PIL-Bilinear.png")
img6.save("PIL-Bicubic.png")
img7.save("PIL-Lanczos.png")


(500, 500, 3)
--- 0.06772160530090332 seconds ---
--- 0.1070241928100586 seconds ---
--- 1.506770133972168 seconds ---
--- 0.00044918060302734375 seconds ---
--- 0.0043027400970458984 seconds ---
--- 0.006240129470825195 seconds ---
--- 0.008253812789916992 seconds ---
1554.4384838262151
107552.67538746337
956.6242731619988
0.0
0.0
0.0
0.0


In [82]:
import time
raw =Image.open(f"./aliasing.png")
img = np.asarray(raw)
ratio = 20
start_time = time.time()
img4 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.NEAREST)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img5 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.BILINEAR)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img6 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.BICUBIC)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

img7 = raw.resize((int(img.shape[1]*ratio), int(img.shape[0]*ratio)), Image.LANCZOS)
print("--- %s seconds ---" % (time.time() - start_time))

# img4 = raw.resize((img.shape[1], img.shape[0]), Image.NEAREST)
# img5 = raw.resize((img.shape[1], img.shape[0]), Image.BILINEAR)
# img6 = raw.resize((img.shape[1], img.shape[0]), Image.BICUBIC)
# img7 = raw.resize((img.shape[1], img.shape[0]), Image.LANCZOS)
print(distance.euclidean(np.asarray(img7).flatten(), np.asarray(img4).flatten()))
print(distance.euclidean(np.asarray(img7).flatten(), np.asarray(img5).flatten()))
print(distance.euclidean(np.asarray(img7).flatten(), np.asarray(img6).flatten()))
print(distance.euclidean(np.asarray(img7).flatten(), np.asarray(img7).flatten()))
img4.save("PIL-Nearest.png")
img5.save("PIL-Bilinear.png")
img6.save("PIL-Bicubic.png")
img7.save("PIL-Lanczos.png")


--- 0.0002474784851074219 seconds ---
--- 0.00029540061950683594 seconds ---
--- 0.00026416778564453125 seconds ---
--- 0.0002532005310058594 seconds ---
0.0
0.0
0.0
0.0
